<a href="https://colab.research.google.com/github/Pjh940114/ch_project_3/blob/master/%EC%A2%85%EB%AA%A9%ED%86%A0%EB%A1%A0%EB%B0%A9_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import pandas as pd
from datetime import datetime
import re
import os
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

In [36]:
# 경로 및 기본설정
start = time.time()

default_path = '/content/drive/MyDrive/krx_nlp/'
default_list = ['lg_chem', 'samsung_sdi', 'sk_innov', 'posco_chem', 'korea_zinc'] # 네이버 다음 주식 이름 통일

naver_raw_list = ['LG화학', '삼성SDI', 'SK이노베이션', '포스코케미칼', '고려아연']
daum_raw_list = ['LGCM', 'SSSDI', 'SKIV', 'POSCOCM', 'KoreaZinc']

naver_to_default = dict(zip(naver_raw_list, default_list))
daum_to_default = dict(zip(daum_raw_list, default_list))

naver = 'naver_board_'
daum = 'daum_talks_'

# stopwords 설정
stopwords = ['삭제된 게시물의 답글', '[자막뉴스]']

# 댓글 달린 수 삭제
for i in range(20):
  stopwords.append(str([i]))

In [37]:
# 파일 이름 포맷 통일
def changefilename(path):
  for k, v in naver_to_default.items():
    os.rename(path + naver + k + '.csv', path + v + '_naver' + '.csv')

  for k, v in daum_to_default.items():
    os.rename(path + daum + k + '.csv', path + v + '_daum' + '.csv')

################ 최초 1회 실행 #################
# changefilename(default_path + 'rawdata/')

In [38]:
# 불러오기
def read_csv(files, path):
  for file in files:
    try:
      globals()[file.split('.')[0]] = pd.read_csv(path + '{}'.format(file), encoding = 'utf-8-sig')
    except:
      print("not read :", file)

read_csv(os.listdir(default_path + 'rawdata/'), default_path + 'rawdata/')

not read : sample_stock_data.csv
not read : .ipynb_checkpoints


In [39]:
def set_df(channel, df):
  print("전처리 전 데이터 길이 :", len(df))
  # df 역순으로 만들기
  r_idx = [i for i in range(df.shape[0]-1, -1, -1)]
  r_df = pd.DataFrame(df, index = r_idx)

  # 인덱스 초기화
  r_df.reset_index(drop = True, inplace = True)
  df = r_df

  # column 제거 및 이름 변경
  if channel == 'naver':
    df.drop(columns = ['글쓴이', '조회', '공감', '비공감'], inplace = True)

  elif channel == 'daum':
    df.drop(columns = ['Unnamed: 0', '종목명'], inplace = True)

    # column 순서 변경
    col1 = df.columns[1:].to_list()
    col2 = df.columns[:1].to_list()
    new_col = col1 + col2
    df = df[new_col]

  # column명 통일
  df.columns = ['Date', 'Title']

  # 날짜 세팅(YYYY-MM-DD)
  df['Date'] = df['Date'].str.replace("[^0-9]", "")

  if channel == 'naver':
    for i in range(len(df['Date'])):
      df['Date'][i] = df['Date'][i][:8]

    # for i in range(len(df)):
    #   df.day[i] = datetime.strptime(str(df.day[i]), '%Y%m%d')

  # if channel == 'daum':
  #   for i in range(len(df)):
  #     df.day[i] = datetime.strptime(str(df.day[i]), '%y%m%d')

  # 날짜에서 시간 제거
  df['Date'] = df['Date'].astype(str).str.replace("00:00:00","")

  # 전처리

  # 특수문자 제거 : 한글, 영어, 숫자 살리고 특수문자 -> replace : ex) +/-/%/↑/↓/...
  df['Title'] = df['Title'].str.replace('[^一-龥ㄱ-힣a-zA-Z0-9\s]+',"",regex=True)
  # df['Title'] = df['Title'].str.replace('[^가-힣\sㄱ-ㅎ0-9+-]+',"",regex=True)
  # df['Title'] = df['Title'].str.replace('^[ㄱ-ㅎ|가-힣|a-z|A-Z|0-9|]*$',"",regex=True)

  # 공백 제거
  df['Title'] = df['Title'].replace('', np.nan) # 자음까지 살림
  df = df.dropna(how = 'any')

  # stopwords 제거
  for stopword in stopwords:
    df['Title'] = df['Title'].str.replace(f"{stopword}","")

  # 제목 중복 제거
  df.drop_duplicates(subset = ['Title'], inplace = True)

  print("전처리 후 데이터 길이 :", len(df))
  print(f"{time.time() - start:.2f} sec")
  return df

In [40]:
lg_chem_naver = set_df('naver', lg_chem_naver)
lg_chem_daum = set_df('daum', lg_chem_daum)
samsung_sdi_naver = set_df('naver', samsung_sdi_naver)
samsung_sdi_daum = set_df('daum', samsung_sdi_daum)
sk_innov_naver = set_df('naver', sk_innov_naver)
sk_innov_daum = set_df('daum', sk_innov_daum)
posco_chem_naver = set_df('naver', posco_chem_naver)
posco_chem_daum = set_df('daum', posco_chem_daum)
korea_zinc_naver = set_df('naver', korea_zinc_naver)
korea_zinc_daum = set_df('daum', korea_zinc_daum)

전처리 전 데이터 길이 : 66620
전처리 후 데이터 길이 : 56799
32.58 sec
전처리 전 데이터 길이 : 573
전처리 후 데이터 길이 : 423
32.62 sec
전처리 전 데이터 길이 : 44084
전처리 후 데이터 길이 : 36681
46.27 sec
전처리 전 데이터 길이 : 313
전처리 후 데이터 길이 : 242
46.30 sec
전처리 전 데이터 길이 : 101211
전처리 후 데이터 길이 : 86396
100.45 sec
전처리 전 데이터 길이 : 288
전처리 후 데이터 길이 : 228
100.48 sec
전처리 전 데이터 길이 : 33161
전처리 후 데이터 길이 : 28309
110.50 sec
전처리 전 데이터 길이 : 120
전처리 후 데이터 길이 : 113
110.52 sec
전처리 전 데이터 길이 : 4303
전처리 후 데이터 길이 : 3907
111.13 sec
전처리 전 데이터 길이 : 44
전처리 후 데이터 길이 : 42
111.15 sec


In [46]:
def get_date(df):
  start_date = '2021-01-01'
  end_date = '2022-06-30'
  # df_filered = df[df['Date'].between(start_date, end_date)]
  try:
    df['Date']=df['Date'].apply(lambda _ : datetime.strptime(_,'%Y%m%d'))
  except:
    pass

  mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
  df = df.loc[mask]
  return df

lg_chem_naver = get_date(lg_chem_naver)
# get_date(lg_chem_daum)
samsung_sdi_naver = get_date(samsung_sdi_naver)
# get_date(samsung_sdi_daum)
sk_innov_naver = get_date(sk_innov_naver)
# get_date(sk_innov_daum)
posco_chem_naver = get_date(posco_chem_naver)
# get_date(posco_chem_daum)
korea_zinc_naver = get_date(korea_zinc_naver)
# get_date(korea_zinc_daum)

In [47]:
# 저장하기
import os
path = '/content/drive/MyDrive/krx_nlp/predata' # 본인 구글 드라이브 계정마다 살짝씩 다를 수도 있음
os.chdir(path)

# lg_chem_daum.to_csv("refined_daum_talks_LG화학.csv", mode='w', encoding = 'utf-8-sig')
# sk_innov_daum.to_csv("refined_daum_talks_SK이노베이션.csv", mode='w', encoding = 'utf-8-sig')
# korea_zinc_daum.to_csv("refined_daum_talks_고려아연.csv", mode='w', encoding = 'utf-8-sig')
# samsung_sdi_daum.to_csv("refined_daum_talks_삼성SDI.csv", mode='w', encoding = 'utf-8-sig')
# posco_chem_daum.to_csv("refined_daum_talks_포스코케미칼.csv", mode='w', encoding = 'utf-8-sig')

samsung_sdi_naver.to_csv("refined_naver_talks_삼성SDI.csv", mode='w', encoding = 'utf-8-sig')
lg_chem_naver.to_csv("refined_naver_talks_LG화학.csv", mode='w', encoding = 'utf-8-sig')
sk_innov_naver.to_csv("refined_naver_talks_SK이노베이션.csv", mode='w', encoding = 'utf-8-sig')
posco_chem_naver.to_csv("refined_naver_talks_포스코케미칼.csv", mode='w', encoding = 'utf-8-sig')
korea_zinc_naver.to_csv("refined_naver_talks_고려아연.csv", mode='w', encoding = 'utf-8-sig')